In [1]:
import ncar_jobqueue

In [2]:
cluster = ncar_jobqueue.NCARCluster(
#    cores=9,  # The number of cores you want
    memory='600GB',  # Amount of memory
#    processes=9,  # How many processes
    walltime='01:00:00',  # Amount of wall time
    project="ncgd0011",  #Project id
)

cluster.scale(75)

/glade/u/home/pmora/.conda/envs/upwelling_post/lib/python3.10/site-packages/dask_jobqueue/core.py:251: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/u/home/pmora/.conda/envs/upwelling_post/lib/python3.10/site-packages/dask_jobqueue/core.py:270: FutureWarning: env_extra has been renamed to job_script_prologue. You are still using it (even if only set to []; please also check config files). If you did not set job_script_prologue yet, env_extra will be respected for now, but it will be removed in a future release. If you already set job_script_prologue, env_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glad

In [3]:
#%matplotlib inline
#%load_ext autoreload
#load_ext watermark


import dask
import distributed
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
#import pandas as pd
import xarray as xr
#import xgcm
import cartopy.crs as cr
from cartopy import feature as cf

mpl.rcParams["savefig.dpi"] = 300
mpl.rcParams["savefig.bbox"] = "tight"
mpl.rcParams["figure.dpi"] = 300

xr.set_options(keep_attrs=True, display_style="html")

#%watermark -iv

In [4]:
client = distributed.Client(cluster)
client

<Client: 'tcp://10.12.206.63:41990' processes=75 threads=150, memory=1.83 TiB>

In [5]:
files = '/glade/campaign/cgd/oce/people/bachman/ETP_1_20_tides/HOURLY/ocean_hourly__*.nc'

In [6]:
DS = xr.open_mfdataset(
    files,
    compat="override",
    data_vars="minimal",
    coords="minimal",
    parallel=True,
    concat_dim="time",
    combine="nested",
    chunks={  "yh":200, "xh":200, },
)[["wind_stress_curl_h"]]
#[["so", "thetao", "uo", "vo", "wo"]]

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [ ]:
DS

In [ ]:
tau = DS['wind_stress_curl']
tau

In [ ]:
A2_tau = tau.sel(xh=np.arange(260,283, 0.05), yh=np.arange(0,17, 0.05), method='nearest')

In [ ]:
A2_tau

In [ ]:
#Sesn_A2_tau = A2_tau.groupby('time.season').mean('time').persist()

In [ ]:
month_A2_tau = A2_tau.groupby('time.month').mean('time').persist()

In [ ]:
month_A2_tau

In [ ]:
month_wind_stress_curl = month_A2_tau.to_netcdf('/glade/scratch/pmora/monthly_wind_stress_curl.nc')

In [ ]:
Sesn_A2_tau.plot.contourf(xh='lon',yh='lat',col='season',col_wrap=2)

In [ ]:
count = 0
fig=plt.figure(figsize=(13,11))
for month in month_A2_tau.month:
             count+=1
             ax=fig.add_subplot(4,3,count,projection=cr.PlateCarree())
             month_A2_tau[month_A2_tau.month==month].plot(cmap="coolwarm_r", cbar_kwargs={"label": ""}, levels = np.linspace(-0.0000010, 0.0000010, 11))
             
             ax.coastlines(resolution='10m')
             ax.add_feature(cf.BORDERS)
             ax.set_extent([-100, -77, 0,17])
             
            
#             gl=ax.gridlines()
#             ax.xlabels_botton=True; ax.ylabels_left=True
#            gl.xformatter = LONGITUDE_FORMATTER
#             gl.yformatter = LATITUDE_FORMATTER

In [ ]:
count = 0
fig=plt.figure(figsize=(13,11))
for month in month_A2_tau.month:
             count+=1
             ax=fig.add_subplot(4,3,count,projection=cr.PlateCarree())
             month_A2_tau[month_A2_tau.month==month].plot(cmap="coolwarm_r", cbar_kwargs={"label": ""}, levels = np.linspace(-0.0000010, 0.0000010, 11))
             
             ax.coastlines(resolution='10m')
             ax.add_feature(cf.BORDERS)
             ax.set_extent([-87, -85, 9,11.5])
             
            
#             gl=ax.gridlines()
#             ax.xlabels_botton=True; ax.ylabels_left=True
#            gl.xformatter = LONGITUD-88, -83, 8,12.5E_FORMATTER
#             gl.yformatter = LATITUDE_FORMATTER

In [1]:
client.close()
cluster.close()

NameError: name 'client' is not defined